In [1]:
from ffcv.writer import DatasetWriter
from ffcv.fields import RGBImageField, IntField, TorchTensorField

from torch.utils.data import Dataset
import os
import torch

OMP: Info #277: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/coc/testnvme/pputta7/mambaforge/envs/ffcv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class OfflineTrajectoryDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.files = [os.path.join(data_dir, file) for file in sorted(os.listdir(data_dir))]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file = self.files[idx]
        data = torch.load(file)
        
        length = data['rgbs'].shape[0]
        padding_size = 500 - length
        
        rgbs = torch.nn.functional.pad(data['rgbs'], (0, 0, 0, padding_size), "constant", 0)
        actions = torch.nn.functional.pad(torch.tensor(data['actions']), (0, padding_size), "constant", 0)
        return rgbs, data['imagegoal'], actions, length

In [3]:
dataset = OfflineTrajectoryDataset("/srv/flash1/pputta7/projects/lm-nav/data/datasets/lmnav/offline_1env_karmesh+clip")

In [5]:
# Pass a type for each data field
write_path = "/srv/flash1/pputta7/projects/lm-nav/data/datasets/lmnav/test.beton"
writer = DatasetWriter(write_path, {
    # Tune options to optimize dataset size, throughput at train-time
    'rgbs': TorchTensorField(dtype=torch.float32,
                             shape=(500, 1024)),
    'imagegoal': TorchTensorField(dtype=torch.float32,
                                  shape=(1024,)),
    'actions': TorchTensorField(dtype=torch.int64,
                                shape=(500,)),
    'length': IntField()
})

# Write dataset
writer.from_indexed_dataset(dataset)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 527/527 [00:02<00:00, 262.69it/s]


In [20]:
out = dataset[0]

In [30]:
out[2].dtype

torch.int64